In [6]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

In [7]:
data_loc = '../data/'

In [8]:
fighters = pd.read_csv(data_loc + 'update_list_fighters.csv')
fighters.tail(3)

,fighter,fighter_url
1343,Josh Parisian,http://ufcstats.com/fighter-details/b5da8f7d05...
1344,Jordan Leavitt,http://ufcstats.com/fighter-details/ac026518a9...
1345,Matt Sayles,http://ufcstats.com/fighter-details/5b8ba8f551...


#### Extracting data to data frame:

In [ ]:
data = []
column_names = ['name','wins','losses','draws',
                'height','weight','reach','stance','dob',
                'slpm','str_acc','sapm','str_def','td_avg','td_acc','td_def','sub_avg',
                'last_update','last_match','url']

for url in tqdm(fighters.fighter_url):
    data.append(get_fighter_data(url=url))

df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

  5%|▍         | 64/1346 [00:30<11:43,  1.82it/s]

#### Merge data with existing file: 

In [9]:
org = pd.read_csv(data_loc + 'ufc_fighters_stats.csv')
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
3950,Jessica Rakoczy,1,5,0 (1 NC),"5'7""",115lbs.,--,Orthodox,"Apr14,1977",2.5,44%,3.55,64%,0.75,20%,0%,0.8,07-04-2023,25-04-2015,http://ufcstats.com/fighter-details/ffdeb4fbea...
3951,David Onama,10,2,0,"5'11""",145lbs.,"74""",Orthodox,"Jun07,1994",5.3,45%,5.75,51%,1.06,20%,45%,0.3,07-04-2023,13-08-2022,http://ufcstats.com/fighter-details/ffe9703408...
3952,Dillon Smith,0,0,0,"6'0""",170lbs.,--,,--,0.0,0%,0.00,0%,0.00,0%,0%,0.0,07-04-2023,25-09-2009,http://ufcstats.com/fighter-details/ffee353f55...


In [10]:
data = pd.concat([org, df]).drop_duplicates(subset=['name']).reset_index(drop=True)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
3945,Jessica Rakoczy,1,5,0 (1 NC),"5'7""",115lbs.,--,Orthodox,"Apr14,1977",2.5,44%,3.55,64%,0.75,20%,0%,0.8,07-04-2023,25-04-2015,http://ufcstats.com/fighter-details/ffdeb4fbea...
3946,David Onama,10,2,0,"5'11""",145lbs.,"74""",Orthodox,"Jun07,1994",5.3,45%,5.75,51%,1.06,20%,45%,0.3,07-04-2023,13-08-2022,http://ufcstats.com/fighter-details/ffe9703408...
3947,Dillon Smith,0,0,0,"6'0""",170lbs.,--,,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,07-04-2023,25-09-2009,http://ufcstats.com/fighter-details/ffee353f55...


#### Saving file:

In [11]:
data.to_csv(data_loc + 'ufc_fighters_stats.csv', index=0)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
3945,Jessica Rakoczy,1,5,0 (1 NC),"5'7""",115lbs.,--,Orthodox,"Apr14,1977",2.5,44%,3.55,64%,0.75,20%,0%,0.8,07-04-2023,25-04-2015,http://ufcstats.com/fighter-details/ffdeb4fbea...
3946,David Onama,10,2,0,"5'11""",145lbs.,"74""",Orthodox,"Jun07,1994",5.3,45%,5.75,51%,1.06,20%,45%,0.3,07-04-2023,13-08-2022,http://ufcstats.com/fighter-details/ffe9703408...
3947,Dillon Smith,0,0,0,"6'0""",170lbs.,--,,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,07-04-2023,25-09-2009,http://ufcstats.com/fighter-details/ffee353f55...


### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>